# 자체 평가지표
* 가지고 있는 제출 파일과 비교

In [8]:
import pandas as pd
from tqdm import tqdm

In [9]:
# 판다스 라이브러리에서 과학적 표기 사용 안하기
pd.options.display.float_format = '{:.5f}'.format
# 리셋 
# pd.reset_option('display.float_format')

In [10]:
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")

### 평가 지표 함수화

In [11]:
def evaluation_index(before_sub, present_sub):
    # 전 최고 제출 파일 열 정리
    before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').to_list()
    before_sub = before_sub.drop('일자|시간|구분', axis=1)
    before_sub = before_sub[['일자', '시간', '구분', '공급량']]
    # 현재 제출 파일 열 정리
    present_sub[['일자', '시간', '구분']] = present_sub['일자|시간|구분'].str.split(' ').to_list()
    present_sub = present_sub.drop('일자|시간|구분', axis=1)
    present_sub = present_sub[['일자', '시간', '구분', '공급량']]

    eval_set = pd.DataFrame(['MSE','RMSE', 'MAE', 'R2','NMAE'], columns=['평가지표'])
    for division in tqdm(present_sub['구분'].unique()):
        temp_before_sub = before_sub[before_sub['구분'] == division]
        temp_present_sub = present_sub[present_sub['구분'] == division]

        # 전 제출 값 = 실제값 취급
        # 현제 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_present_sub['공급량']
        y_mean = y.mean()

        # MSE
        mse = ( ( y - pred) ** 2 ).sum() / len(pred)
            
        # RMSE
        rmse = mse ** (1/2)
        
        # MAE
        mae = ( abs( y - pred ) ).sum() / len(pred)
        
        # R2
        upper = ( (y - pred) ** 2 ).sum()
        lower = ( (y - y_mean) ** 2 ).sum()
        if lower == 0:
            fraction = 0
        else:
            fraction = upper / lower
        r2 = 1 - fraction
        
        #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
        nmae = ( abs( y - pred ) / y ).sum() / len(pred)
        
        # 평가지표 리스트
        one_eval_set = [mse, rmse, mae, r2, nmae]
        
        # 평가지표 셋에 담기
        eval_set[division] = one_eval_set

    # 평가지표 평균 구하기
    eval_set['평가지표_평균'] = eval_set.mean(axis=1)

    # 평가지표 평균 출력
    print(eval_set[['평가지표', '평가지표_평균']])
    
    return eval_set

## NAE 평가 지표 셋 함수화

In [12]:
def nae_set(before_sub, present_sub):
    # 일자 시간을 한번에 지정할 수 있게 편집
    before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').tolist()
    before_sub['일자|시간'] = before_sub['일자'] + ' ' + before_sub['시간']
    before_sub['예측공급량'] = present_sub['공급량']

    eval_set = pd.DataFrame()
    # 예측 값 구분별로 평가하기
    for division in tqdm(before_sub['구분'].unique()):
        # 구분별로 데이터 가져오기
        temp_before_sub = before_sub[before_sub['구분'] == division]

        # 전 최고 제출 값 = 실제값 취급
        # 현재 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_before_sub['예측공급량']
        
        #NAE(Normalized Absolute Error)-정규화 절대 오차 척도(NMAE에서 평균을 취하지 않음)
        temp_before_sub['NAE'] = abs( y - pred ) / y 
        
        # 평가지표 셋에 담기
        eval_set = pd.concat([eval_set, temp_before_sub], ignore_index=True)
    
        # 평가지표 셋의 열 정리
        eval_set = eval_set[['일자|시간', '구분', '공급량', '예측공급량', 'NAE']]
    return eval_set

## NAE 조건에 따른 샘플 시각화 

In [13]:
def vis_nae(res_df, res_num):
    # 얼마 이상의 NAE 샘플을 설정
    num = res_num
    res_df = all_nae[all_nae['NAE'] >= num]
    print(f"총 NAE {num}이상 샘플 개수 :",len(res_df['NAE']))

    # 구분별로 그래프(총 구분 7개 : 'A', 'B', 'C', 'D', 'E', 'G', 'H')
    for div in tqdm(res_df['구분'].unique()):
        temp = res_df[res_df['구분']==div]
        print(f"구분 {div}의 NAE {num}이상 샘플 개수 :",len(temp['NAE']))
        
        plt.figure(figsize=(16,10))
        plt.title(f'구분 : {div} NAE : {num}이상', fontdict={'size':20})
        plt.xticks(fontsize=15, rotation=90)
        plt.yticks(fontsize=15)
        x = temp['일자|시간']
        y1 = temp['공급량']
        y2 = temp['예측공급량']
        plt.plot(x, y1, 'b', label='공급량')
        plt.plot(x, y2, 'r', label='예측공급량')
        plt.legend(loc='upper right', fontsize=15)

# 데이터 가져오기

In [14]:
# team_sub_ver0.4.csv

# 최고점 제출 파일
before_sub = pd.read_csv('../08_종합/team_sub_ver0.4.csv')
# 현재 제출 파일
present_sub = pd.read_csv('../07_결과_출력/pycaret_robust_transformation_blender.csv')

# 평가지표 평균 출력

In [15]:
# 구분별 계산된 평가지표 셋 = evaluation_index(최고점 제출 셋, 현재 제출 셋)
set = evaluation_index(before_sub, present_sub)

100%|██████████| 7/7 [00:00<00:00, 45.28it/s]

   평가지표   평가지표_평균
0   MSE 216.24319
1  RMSE  12.55097
2   MAE   9.50348
3    R2   0.99871
4  NMAE   0.00640


In [16]:
set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,300.81428,165.52735,2.73255,87.80110,443.09622,503.96677,9.76408,216.24319
1,RMSE,17.34400,12.86574,1.65304,9.37022,21.04985,22.44920,3.12475,12.55097
2,MAE,13.13433,10.00198,1.29681,7.18628,15.86643,16.61302,2.42553,9.50348
3,R2,0.99872,0.99915,0.99735,0.99887,0.99880,0.99887,0.99922,0.99871
4,NMAE,0.00687,0.00631,0.00714,0.00656,0.00678,0.00543,0.00572,0.00640


# NAE 셋 출력

In [17]:
# NAE값 계산된 셋 = nae_set(최고점 제출 셋, 현재 제출 셋)
all_nae = nae_set(before_sub, present_sub)
all_nae.head()

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-12-3c612ab38485>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_before_sub['NAE'] = abs( y - pred ) / y
100%|██████████| 7/7 [00:00<00:00, 118.96it/s]


,일자|시간,구분,공급량,예측공급량,NAE
0,2019-01-01 01,A,1949.27983,1969.20170,0.01022
1,2019-01-01 02,A,1833.49026,1800.89594,0.01778
2,2019-01-01 03,A,1744.56689,1741.78783,0.00159
3,2019-01-01 04,A,1756.34447,1742.54084,0.00786
4,2019-01-01 05,A,1889.02308,1897.16565,0.00431


## NAE 특정 조건 이상 그래프 시각화

In [21]:
# vis_nae(NAE 셋, NAE 조건 값(NAE >= 값))
# NAE 0.05이상인 샘플 시각화
vis_nae(all_nae, 0.05)

0it [00:00, ?it/s]

총 NAE 0.05이상 샘플 개수 : 0
